# Conceptual Analysis Workbook for 6/15
### Plan:
* Load models (GPT-3 from txt, OPT from transformers)
* Define helper methods for angle tests
* Conduct angle tests w/ GPT and OPT
* Define + discuss complexity tests
* Conduct complexity tests w/ OPT

In [1]:
import os
import openai
import torch

import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from transformers import GPT2Tokenizer, OPTModel

# Set in conda
openai.api_key = os.getenv('OPENAI_API_KEY')

## Load Models

### Vocab

In [2]:
# Load our common words vocab from file

vocab = []
with open(u'/gpfs/fs1/home/mbarlow6/Desktop/Conceptual-Analysis/barlow/valid_vocab.txt', 'r') as f:
    for w in f:
        vocab.append(w.strip())

### GPT-3

In [3]:
# Loading saved embeddings from GPT-3 Ada
# Ada embeds: 1024 dims

ada_embeds = []
with open(u'/gpfs/fs1/home/mbarlow6/Desktop/Conceptual-Analysis/barlow/gpt/gpt_ada.txt', 'r') as f:
    for line in f:
        ada_embeds.append([float(x) for x in line.strip().split()])

model_gpt = dict(zip(vocab, ada_embeds))

# Babbage load included below as comment.
"""
# Babbage embeds: 2048 dims
bab_embeds = []
with open(u'/gpfs/fs1/home/mbarlow6/Desktop/Conceptual-Analysis/barlow/gpt/gpt_babbage.txt', 'r') as f:
    for line in f:
        bab_embeds.append([float(x) for x in line.strip().split()])

model_bab = dict(zip(vocab, bab_embeds))
"""
#

"\n# Babbage embeds: 2048 dims\nbab_embeds = []\nwith open(u'/gpfs/fs1/home/mbarlow6/Desktop/Conceptual-Analysis/barlow/gpt/gpt_babbage.txt', 'r') as f:\n    for line in f:\n        bab_embeds.append([float(x) for x in line.strip().split()])\n\nmodel_bab = dict(zip(vocab, bab_embeds))\n"

In [4]:
# this will be my function for getting GPT-3 embeddings from API
gpt_models = ['text-similarity-ada-001',
              'text-similarity-babbage-001',
              'text-similarity-curie-001',
              'text-similarity-davinci-001']
def get_embedding(text, engine='text-similarity-ada-001'):
    text = text.replace('\n', ' ')
    return openai.Embedding.create(input=[text], engine=engine)['data'][0]['embedding']

### OPT

In [5]:
# Loding OPT with HuggingFace Transformers. Paramenter size can be changed- ['125m', '350m', '1.3b', '2.7b', '6.7b', '13b', '30b']
# NOTE: 1.3b uses 2.6G of .cache space.

# NOTE: The documentation recommends using the GPT2Tokenizer here.
tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-1.3b", cache_dir='/scratch/mbarlow6/.cache/')
model_opt_raw = OPTModel.from_pretrained("facebook/opt-1.3b", cache_dir='/scratch/mbarlow6/.cache/')

In [6]:
opt_embeds = []
with open(u'/gpfs/fs1/home/mbarlow6/Desktop/Conceptual-Analysis/barlow/opt/1_3B.txt', 'r') as f:
    for line in f:
        opt_embeds.append([float(x) for x in line.strip().split()])
model_opt = dict(zip(vocab, opt_embeds))

In [20]:
# This is how I'm getting the embedding from OPT.
def squeeze(word, tokenizer=tokenizer, model=model_opt_raw, debug=False):
    # prepare inputs and model
    inputs = tokenizer(word, return_tensors="pt") # return pytorch tensors
    if debug:
        print('Tokens Requested:')
        print(tokenizer.batch_decode(inputs.input_ids[0]))
    with torch.no_grad(): # added 6/16
        outputs = model(**inputs)

    # get embedding from final layer
    last_hidden_states = outputs.last_hidden_state # I have confirmed this is the same
                                                   # As the strategy from Tianyi

    # remove unnecessary dimension
    embeddings = torch.squeeze(last_hidden_states, dim=0)

    # convert to list
    # return [embed.tolist() for embed in embeddings][1]

    # add all of the vectors together
    final_embed = np.array(embeddings[1])
    if embeddings.shape[0] > 2:
        for next_emb in embeddings[2:]:
            final_embed += np.array(next_emb)
        final_embed /= embeddings.shape[0] - 1
    return final_embed

In [8]:
squeeze('Hello world! It is I, Don Quixote.')

Tokens Requested:
['</s>', 'Hello', ' world', '!', ' It', ' is', ' I', ',', ' Don', ' Qu', 'ix', 'ote', '.']


array([-1.1419824 , -0.64336497, -1.0564834 , ...,  0.84003204,
       -1.6957244 , -1.0830795 ], dtype=float32)

## Define Helpers (for angle)

In [9]:
def positive(words, model='gpt'):
    """
    Args:
        words: iterable
        model: 'gpt' or 'opt'
    Returns:
        Positive (summed vectors) of word embeddings of a given list of words from the specified model. Defaults to GPT-3.
    """
    if isinstance(words, str):
        print(f"You requested the positive of the string \"{words}\". Did you mean [\"{words}\"]?")

    out = 0
    for token in words:
        # convert token to string
        word = str(token)

        # do model check - least intensive operation to repeat
        if model.lower() == 'gpt':
            # look for token in cached GPT embeds
            if word in model_gpt:
                ex = model_gpt[word]  # ex for "extracted"
            # if not found, query API
            else:
                ex = get_embedding(word)
                model_gpt[word] = ex
        elif model.lower() == 'opt':
            if word in model_opt:
                ex = model_opt[word]
            else:
                # squeeze!
                ex = squeeze(word)
        else:
            raise ValueError('Please provide either gpt or opt as a model choice.')

        # construct positive
        if isinstance(out, int):
            out = np.array(ex).reshape(1, -1)
        else:
            out += np.array(ex).reshape(1, -1)
            
    return out if not isinstance(out, int) else np.array([])

In [26]:
def most_similar(target, phrase=False, topn=10, model='gpt'):
    """
    Args:
        target: str or list[float] (if phrase)
        phrase: bool    -> True if target is vector. Default False.
        topn: int       -> Number of results to keep [0:2+topn]
        model: 'gpt' or 'opt'
    Returns:
        Top n most similar words in common vocab to the target vector in the specified model.
    """
    # holder for all similarity scores
    sims = {}

    # initialize starting point
    a = target

    # if no phrase, must convert to embed.
    if not phrase:
        a = positive([a], model) # I'm going to abuse positive.

    # now, we check every B in our vocab
    for term in vocab:
        b = positive([term], model)
        if isinstance(target, str):
            if term != target:
                sims[term] = cosine_similarity(a, b)[0][0]
        elif b.any():
            sims[term] = cosine_similarity(a, b)[0][0]
    
    return list({k: v for k, v in sorted(sims.items(), key=lambda x: x[1], reverse=True)}.items())[:2+topn]

In [11]:
def definition(words, topn=10, model='gpt'):
    """
    Args:
        words: iterable -> Items to be combined into a positive and defined
        topn: int       -> number of words to keep in definiton
        model: 'gpt' or 'opt'
    Returns:
        Top N most similar words in the specified model to a positive of the string representations
        of any provided interable, if it's members can be cast to strings. 
    """
    # get phrase
    if isinstance(words, str):
        words = [words]
    phrase = positive(words, model)

    # get definition
    def_of_positive = most_similar(phrase, True, topn, model)

    return def_of_positive

In [38]:
def define(words, topn=10):
    """
    Args:
        words: string or iterable -> Items to be combined into a positive and defined
        topn: int                 -> number of words to keep in definiton
    Returns:
        None. Formats and prints definition from both models.
    """
    print(*words, sep=', ')
    print('GPT-3:')
    print(definition(words, topn, 'gpt'))
    print('------------------------------------------------------')
    print('OPT:')
    print(definition(words, topn, 'opt'))
    print('------------------------------------------------------')

In [13]:
def calculate_similarity(words, target, vec=False, model='gpt'):
    """
    Args:
        words: iterable or vector   -> Items to be combined into a positive and compared,
                                       or already constructed vector of matching dimensions.
        target: str     -> single term to calulate similarity to.
        vec: bool       -> true if 'words' is a vector
        model: 'gpt' or 'opt'
    Returns:
        The cosine similarity of the words vector and target term in specified model.
    """
    # get phrase
    phrase = words
    if not vec:
        if isinstance(words, str):
            phrase = positive([words], model)
        else:
            phrase = positive(words, model)
    
    # get target
    target = positive([target], model)

    return cosine_similarity(phrase, target)[0][0]


In [14]:
def sim_test(words, target):
    """
    Args:
        words: iterable, str, or vector -> Items to be combined into a positive and compared,
                                            or already constructed vector of matching dimensions.
        target: str                     -> single term to calulate similarity to.
    Returns:
        None. Formats and prints similarity from both models.
    """
    print(*words, sep=', ', end='')
    print(f' -> {target}')
    print('GPT-3:')
    print(calculate_similarity(words, target, model='gpt'))
    print('------------------------------------------------------')
    print('OPT:')
    print(calculate_similarity(words, target, model='opt'))
    print('------------------------------------------------------')

I'm choosing to leave out a function where both A and B are vectors, as that is mostly helpful for phrase vectors we're not constucting ourselves - so I'll include it later if needed.

## Conduct Angle tests

### A - Bachelors, Husbands, and Wives
Words describing spousal relations.

In [185]:
define(['man', 'unmarried'])

man, unmarried
GPT-3:
[('man', array([[0.96075165]])), ('unmarried', array([[0.96075165]])), ('mixture', array([[0.95492556]])), ('sort', array([[0.92664048]])), ('resident', array([[0.92474028]])), ('kitchen', array([[0.91976509]])), ('landscape', array([[0.91883643]])), ('occupy', array([[0.91856791]])), ('geography', array([[0.91596578]])), ('knit', array([[0.91542595]])), ('p.m.', array([[0.91415888]])), ('railroad', array([[0.91380011]]))]
------------------------------------------------------
OPT:
[('man', array([[0.88599687]])), ('manner', array([[0.84875869]])), ('manage', array([[0.8198654]])), ('unmarried', array([[0.76266626]])), ('unfortunately', array([[0.73278264]])), ('unemployed', array([[0.73142168]])), ('boyfriend', array([[0.71435011]])), ('unlikely', array([[0.70675865]])), ('unlike', array([[0.69843259]])), ('woman', array([[0.6917361]])), ('mate', array([[0.68681759]])), ('unacceptable', array([[0.68536389]]))]
-----------------------------------------------------

In [186]:
define(['man', 'married'])

man, married
GPT-3:
[('man', array([[0.96178941]])), ('married', array([[0.9617894]])), ('mixture', array([[0.95722058]])), ('palace', array([[0.92757294]])), ('tax', array([[0.92669628]])), ('earn', array([[0.92401706]])), ('package', array([[0.92374845]])), ('phase', array([[0.92117782]])), ('pupil', array([[0.92108494]])), ('ease', array([[0.91982054]])), ('cancel', array([[0.91770757]])), ('painful', array([[0.91756225]]))]
------------------------------------------------------
OPT:
[('man', array([[0.86633847]])), ('manner', array([[0.83163956]])), ('manage', array([[0.81410604]])), ('married', array([[0.7982615]])), ('boyfriend', array([[0.74501651]])), ('mate', array([[0.71873402]])), ('yesterday', array([[0.69999539]])), ('woman', array([[0.697068]])), ('brother', array([[0.69481484]])), ('brothers', array([[0.69035788]])), ('girl', array([[0.68302403]])), ('yeah', array([[0.67783933]]))]
------------------------------------------------------


In [187]:
sim_test(['man', 'unmarried'], 'bachelor')

man, unmarried -> bachelor
GPT-3:
0.8694018001827069
------------------------------------------------------
OPT:
0.41610598788816566
------------------------------------------------------


In [188]:
sim_test(['man', 'married'], 'husband')

man, married -> husband
GPT-3:
0.8948082387821381
------------------------------------------------------
OPT:
0.6647823236771563
------------------------------------------------------


In [189]:
define(['woman', 'married'])

woman, married
GPT-3:
[('woman', array([[0.96390753]])), ('married', array([[0.96390753]])), ('pupil', array([[0.93324007]])), ('railroad', array([[0.92834275]])), ('disappointment', array([[0.92653557]])), ('cancel', array([[0.9251154]])), ('painful', array([[0.92502677]])), ('palace', array([[0.92372286]])), ('fetch', array([[0.92271516]])), ('faith', array([[0.92236675]])), ('running', array([[0.92115944]])), ('actor', array([[0.92062613]]))]
------------------------------------------------------
OPT:
[('woman', array([[0.88587642]])), ('married', array([[0.82386254]])), ('daughter', array([[0.77974624]])), ('boyfriend', array([[0.76577424]])), ('girlfriend', array([[0.74751902]])), ('girl', array([[0.73701611]])), ('wife', array([[0.73488944]])), ('husband', array([[0.73301067]])), ('grandmother', array([[0.68714716]])), ('granddaughter', array([[0.68301617]])), ('children', array([[0.67936361]])), ('manner', array([[0.67808685]]))]
-------------------------------------------------

In [190]:
sim_test(['woman', 'married'], 'wife')

woman, married -> wife
GPT-3:
0.8418697738514673
------------------------------------------------------
OPT:
0.734889438201916
------------------------------------------------------


In [191]:
define(['woman', 'spouse'])

woman, spouse
GPT-3:
[('spouse', array([[0.94943401]])), ('woman', array([[0.949434]])), ('width', array([[0.92651648]])), ('railroad', array([[0.91887132]])), ('painful', array([[0.91856077]])), ('faith', array([[0.91742113]])), ('cancel', array([[0.91710516]])), ('palace', array([[0.91570877]])), ('critical', array([[0.91506279]])), ('till', array([[0.91293668]])), ('lazy', array([[0.91244314]])), ('beside', array([[0.91200722]]))]
------------------------------------------------------
OPT:
[('woman', array([[0.91228447]])), ('spouse', array([[0.78908906]])), ('boyfriend', array([[0.77552945]])), ('spouses', array([[0.77229732]])), ('daughter', array([[0.76964035]])), ('husband', array([[0.76885478]])), ('wife', array([[0.75410773]])), ('girlfriend', array([[0.74503148]])), ('girl', array([[0.74202005]])), ('moment', array([[0.70004672]])), ('manner', array([[0.69585122]])), ('guy', array([[0.69518836]]))]
------------------------------------------------------


In [192]:
define(['women', 'spouses'])

women, spouses
GPT-3:
[('spouses', array([[0.96412527]])), ('spring', array([[0.93702292]])), ('ear', array([[0.93473576]])), ('party', array([[0.93251428]])), ('ours', array([[0.92566306]])), ('brush', array([[0.92565899]])), ('painful', array([[0.92550278]])), ('various', array([[0.92221346]])), ('geography', array([[0.92187047]])), ('sisters', array([[0.92024253]])), ('access', array([[0.92019819]])), ('empire', array([[0.91934091]]))]
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'women']
[('spouses', array([[0.837883]])), ('children', array([[0.79607431]])), ('girls', array([[0.78910444]])), ('wives', array([[0.78439078]])), ('husbands', array([[0.73374251]])), ('woman', array([[0.73218368]])), ('boys', array([[0.7312016]])), ('people', array([[0.71320289]])), ('spouse', array([[0.70230903]])), ('daughter', array([[0.69744422]])), ('sheep', array([[0.69038891]])), ('himself', array([[0.68383807]]))]
-----------------------------------------

In [193]:
sim_test(['woman', 'spouse'], 'wife')

woman, spouse -> wife
GPT-3:
0.8302057291267502
------------------------------------------------------
OPT:
0.7541077283467653
------------------------------------------------------


In [194]:
sim_test(['women', 'spouses'], 'wives')

women, spouses -> wives
GPT-3:
0.8865878369668883
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'women']
0.7843907847361444
------------------------------------------------------


In [195]:
define(['man', 'spouse'])

man, spouse
GPT-3:
[('mixture', array([[0.95253434]])), ('spouse', array([[0.94911784]])), ('man', array([[0.94911783]])), ('virtually', array([[0.9221998]])), ('silver', array([[0.91915238]])), ('information', array([[0.91893894]])), ('separate', array([[0.91803186]])), ('palace', array([[0.91785384]])), ('flavour', array([[0.91667311]])), ('till', array([[0.91661684]])), ('width', array([[0.9162023]])), ('occupy', array([[0.91355709]]))]
------------------------------------------------------
OPT:
[('man', array([[0.89942324]])), ('manner', array([[0.85772565]])), ('manage', array([[0.81622573]])), ('spouse', array([[0.76242521]])), ('boyfriend', array([[0.74822593]])), ('spouses', array([[0.73851195]])), ('mate', array([[0.70810371]])), ('woman', array([[0.70309464]])), ('boy', array([[0.69514694]])), ('husband', array([[0.69127019]])), ('moment', array([[0.68736434]])), ('girl', array([[0.67885692]]))]
------------------------------------------------------


In [196]:
define(['men', 'spouses'])

men, spouses
GPT-3:
[('spouses', array([[0.96002072]])), ('party', array([[0.93883412]])), ('box', array([[0.93116393]])), ('brush', array([[0.92933049]])), ('ours', array([[0.92874469]])), ('painful', array([[0.92799058]])), ('various', array([[0.92785933]])), ('fetch', array([[0.92750024]])), ('geography', array([[0.92634868]])), ('temporarily', array([[0.92598712]])), ('detailed', array([[0.92587874]])), ('faith', array([[0.92532543]]))]
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'men']
[('spouses', array([[0.84660808]])), ('spouse', array([[0.74563118]])), ('husbands', array([[0.73599219]])), ('boys', array([[0.73183862]])), ('wives', array([[0.71008121]])), ('girls', array([[0.70893027]])), ('children', array([[0.68173138]])), ('siblings', array([[0.67303802]])), ('daughters', array([[0.66850371]])), ('moment', array([[0.66378441]])), ('husband', array([[0.65396775]])), ('offspring', array([[0.65279736]]))]
------------------------------

In [197]:
define(['male', 'spouse'])

male, spouse
GPT-3:
[('spouse', array([[0.94819066]])), ('male', array([[0.94819065]])), ('width', array([[0.92843548]])), ('p.m.', array([[0.92446656]])), ('railroad', array([[0.92237943]])), ('cancel', array([[0.92166726]])), ('faith', array([[0.92109198]])), ('running', array([[0.91976917]])), ('geography', array([[0.91698051]])), ('fetch', array([[0.91675896]])), ('painful', array([[0.91674291]])), ('detailed', array([[0.91596848]]))]
------------------------------------------------------
OPT:
[('male', array([[0.87326338]])), ('female', array([[0.80014056]])), ('spouse', array([[0.77356672]])), ('spouses', array([[0.73729638]])), ('boyfriend', array([[0.67644408]])), ('wife', array([[0.65842795]])), ('husband', array([[0.6539461]])), ('daughter', array([[0.65338688]])), ('woman', array([[0.6446877]])), ('employee', array([[0.64211625]])), ('girlfriend', array([[0.64110653]])), ('girl', array([[0.63088405]]))]
------------------------------------------------------


In [198]:
define(['male', 'spouses'])

male, spouses
GPT-3:
[('spouses', array([[0.95148224]])), ('male', array([[0.95148224]])), ('party', array([[0.9303511]])), ('geography', array([[0.92946202]])), ('fetch', array([[0.92698467]])), ('p.m.', array([[0.92653765]])), ('railroad', array([[0.92604905]])), ('running', array([[0.92502615]])), ('faith', array([[0.92343408]])), ('cancel', array([[0.92161899]])), ('painful', array([[0.92026344]])), ('ours', array([[0.92018114]]))]
------------------------------------------------------
OPT:
[('male', array([[0.86333014]])), ('spouses', array([[0.79540057]])), ('female', array([[0.79339229]])), ('spouse', array([[0.73837002]])), ('husbands', array([[0.68592647]])), ('boyfriend', array([[0.68439661]])), ('daughter', array([[0.67251993]])), ('wives', array([[0.67086208]])), ('woman', array([[0.65861696]])), ('herself', array([[0.65802554]])), ('boys', array([[0.65589311]])), ('girls', array([[0.65273743]]))]
------------------------------------------------------


In [199]:
sim_test(['male', 'spouse'], 'husband')

male, spouse -> husband
GPT-3:
0.8915628588982125
------------------------------------------------------
OPT:
0.6539461025975769
------------------------------------------------------


In [200]:
sim_test(['male', 'spouses'], 'husbands')

male, spouses -> husbands
GPT-3:
0.8687174186239782
------------------------------------------------------
OPT:
0.6859264736097604
------------------------------------------------------


In [201]:
sim_test(['man', 'spouse'], 'husband')

man, spouse -> husband
GPT-3:
0.8939715264899357
------------------------------------------------------
OPT:
0.691270186406268
------------------------------------------------------


In [202]:
sim_test(['men', 'spouses'], 'husbands')

men, spouses -> husbands
GPT-3:
0.8658772325352551
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'men']
0.7359921873260872
------------------------------------------------------


### B - Brothers, sisters, and daughters
Family words related to children.

* Male, sibling, brother.

In [203]:
define(['man', 'sibling'])

man, sibling
GPT-3:
[('sibling', array([[0.96017968]])), ('man', array([[0.96017968]])), ('mixture', array([[0.93901828]])), ('priority', array([[0.91530786]])), ('strip', array([[0.91235929]])), ('diary', array([[0.9096537]])), ('sheet', array([[0.9095486]])), ('infect', array([[0.90333015]])), ('use', array([[0.9018139]])), ('clean', array([[0.90094298]])), ('supply', array([[0.89903186]])), ('mile', array([[0.8990293]]))]
------------------------------------------------------
OPT:
[('man', array([[0.89419924]])), ('manner', array([[0.85168694]])), ('manage', array([[0.80377161]])), ('siblings', array([[0.74284579]])), ('sibling', array([[0.72680261]])), ('suspect', array([[0.72540785]])), ('sister', array([[0.71889461]])), ('boyfriend', array([[0.712106]])), ('sadly', array([[0.71098774]])), ('sisters', array([[0.71072679]])), ('boy', array([[0.69644699]])), ('suddenly', array([[0.6834409]]))]
------------------------------------------------------


In [204]:
sim_test(['man', 'sibling'], 'brother')

man, sibling -> brother
GPT-3:
0.8396088279513378
------------------------------------------------------
OPT:
0.6679592762596386
------------------------------------------------------


In [205]:
sim_test(['men', 'siblings'], 'brothers')

men, siblings -> brothers
GPT-3:
0.8410352841652213
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'men']
0.6497953526607885
------------------------------------------------------


In [206]:
define(['boys', 'siblings'])

boys, siblings
GPT-3:
[('siblings', array([[0.95944378]])), ('boys', array([[0.95944378]])), ('attach', array([[0.94378946]])), ('sufficiently', array([[0.94127007]])), ('painful', array([[0.9228015]])), ('palace', array([[0.92166]])), ('deeply', array([[0.91670334]])), ('lazy', array([[0.91661007]])), ('faith', array([[0.91613933]])), ('musician', array([[0.91588915]])), ('millimetre', array([[0.9156872]])), ('reach', array([[0.91548756]]))]
------------------------------------------------------
OPT:
[('boys', array([[0.91449954]])), ('siblings', array([[0.87065827]])), ('girls', array([[0.83958621]])), ('sisters', array([[0.82475289]])), ('boyfriend', array([[0.7480489]])), ('brothers', array([[0.73731182]])), ('children', array([[0.72503196]])), ('wives', array([[0.72172919]])), ('husbands', array([[0.72062511]])), ('sister', array([[0.7067407]])), ('daughters', array([[0.70424283]])), ('singer', array([[0.69709095]]))]
------------------------------------------------------


In [207]:
sim_test(['boy', 'sibling'], 'brother')

boy, sibling -> brother
GPT-3:
0.8409528303692209
------------------------------------------------------
OPT:
0.6601419017511898
------------------------------------------------------


In [208]:
sim_test(['boys', 'siblings'], 'brothers')

boys, siblings -> brothers
GPT-3:
0.847101667333075
------------------------------------------------------
OPT:
0.737311816255271
------------------------------------------------------


In [209]:
define(['male', 'sibling'])

male, sibling
GPT-3:
[('sibling', array([[0.93807161]])), ('male', array([[0.93807161]])), ('running', array([[0.91181414]])), ('fetch', array([[0.9080858]])), ('railroad', array([[0.90697853]])), ('faith', array([[0.90686211]])), ('cancel', array([[0.90676692]])), ('geography', array([[0.90671527]])), ('promptly', array([[0.90480561]])), ('p.m.', array([[0.90445498]])), ('main', array([[0.90416444]])), ('affect', array([[0.90355422]]))]
------------------------------------------------------
OPT:
[('male', array([[0.87198459]])), ('female', array([[0.79518543]])), ('sibling', array([[0.75141468]])), ('siblings', array([[0.70507791]])), ('sister', array([[0.67325145]])), ('suspect', array([[0.67135762]])), ('sisters', array([[0.66620635]])), ('silly', array([[0.64052713]])), ('sadly', array([[0.62848466]])), ('sensible', array([[0.6282936]])), ('boyfriend', array([[0.62764548]])), ('singer', array([[0.62542285]]))]
------------------------------------------------------


In [210]:
sim_test(['male', 'sibling'], 'brother')

male, sibling -> brother
GPT-3:
0.8541904415388808
------------------------------------------------------
OPT:
0.5604805400965658
------------------------------------------------------


In [211]:
sim_test(['male', 'siblings'], 'brothers')

male, siblings -> brothers
GPT-3:
0.831681511295733
------------------------------------------------------
OPT:
0.6768129077329091
------------------------------------------------------


* Female, sibling, sister

In [212]:
define(['female', 'sibling'])

female, sibling
GPT-3:
[('female', array([[0.94894142]])), ('sibling', array([[0.94894142]])), ('deep', array([[0.92549443]])), ('painful', array([[0.918009]])), ('cancel', array([[0.9160146]])), ('promptly', array([[0.91476622]])), ('palace', array([[0.91425834]])), ('affect', array([[0.91366803]])), ('primary', array([[0.9127056]])), ('running', array([[0.91260407]])), ('upset', array([[0.91226773]])), ('supply', array([[0.91106401]]))]
------------------------------------------------------
OPT:
[('female', array([[0.86960456]])), ('male', array([[0.83729849]])), ('sibling', array([[0.72298015]])), ('siblings', array([[0.68569697]])), ('sister', array([[0.66188809]])), ('sisters', array([[0.64652272]])), ('silly', array([[0.64212161]])), ('suspect', array([[0.63803801]])), ('sadly', array([[0.63451318]])), ('daughter', array([[0.62384798]])), ('girl', array([[0.62339667]])), ('boyfriend', array([[0.61938215]]))]
------------------------------------------------------


In [213]:
define(['female', 'siblings'])

female, siblings
GPT-3:
[('siblings', array([[0.96418032]])), ('female', array([[0.96418032]])), ('attach', array([[0.94840848]])), ('deep', array([[0.94007258]])), ('painful', array([[0.9375789]])), ('cancel', array([[0.93415229]])), ('palace', array([[0.93217046]])), ('automatic', array([[0.92539383]])), ('reach', array([[0.92490965]])), ('critical', array([[0.92458128]])), ('running', array([[0.92398693]])), ('p.m.', array([[0.92317658]]))]
------------------------------------------------------
OPT:
[('female', array([[0.84995164]])), ('male', array([[0.81778721]])), ('siblings', array([[0.77870285]])), ('sisters', array([[0.70990269]])), ('daughter', array([[0.70557974]])), ('girls', array([[0.6888967]])), ('boyfriend', array([[0.68582654]])), ('brothers', array([[0.6847739]])), ('sadly', array([[0.67738579]])), ('sibling', array([[0.67508822]])), ('herself', array([[0.67210598]])), ('sister', array([[0.67026154]]))]
------------------------------------------------------


In [214]:
sim_test(['female', 'sibling'], 'sister')

female, sibling -> sister
GPT-3:
0.8577400040210704
------------------------------------------------------
OPT:
0.6618880883028158
------------------------------------------------------


In [215]:
sim_test(['female', 'siblings'], 'sisters')

female, siblings -> sisters
GPT-3:
0.8599560310184233
------------------------------------------------------
OPT:
0.7099026930323483
------------------------------------------------------


In [216]:
define(['woman', 'sibling'])

woman, sibling
GPT-3:
[('sibling', array([[0.95074864]])), ('woman', array([[0.95074863]])), ('use', array([[0.90781291]])), ('criticize', array([[0.90627842]])), ('disappointment', array([[0.90514363]])), ('convention', array([[0.90302389]])), ('supply', array([[0.90275653]])), ('promptly', array([[0.90171579]])), ('affect', array([[0.90120201]])), ('writing', array([[0.89982994]])), ('initially', array([[0.89950243]])), ('sight', array([[0.89904202]]))]
------------------------------------------------------
OPT:
[('woman', array([[0.89587637]])), ('boyfriend', array([[0.75847323]])), ('siblings', array([[0.75766839]])), ('daughter', array([[0.75735476]])), ('girl', array([[0.74797687]])), ('sibling', array([[0.72322071]])), ('husband', array([[0.7106164]])), ('sisters', array([[0.70624838]])), ('sister', array([[0.70586254]])), ('girlfriend', array([[0.70511143]])), ('manner', array([[0.70042037]])), ('guy', array([[0.68955852]]))]
----------------------------------------------------

In [217]:
define(['women', 'siblings'])

women, siblings
GPT-3:
[('siblings', array([[0.96022151]])), ('spring', array([[0.94392774]])), ('attach', array([[0.94335247]])), ('ear', array([[0.93385946]])), ('painful', array([[0.92785258]])), ('palace', array([[0.92506742]])), ('project', array([[0.92327227]])), ('detailed', array([[0.92300329]])), ('empire', array([[0.92207786]])), ('cancel', array([[0.92117149]])), ('lazy', array([[0.92042697]])), ('south', array([[0.91886338]]))]
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'women']
[('siblings', array([[0.82980536]])), ('girls', array([[0.81353281]])), ('children', array([[0.80043451]])), ('boys', array([[0.7607112]])), ('wives', array([[0.75463091]])), ('sisters', array([[0.75351682]])), ('woman', array([[0.7243997]])), ('daughter', array([[0.72020618]])), ('husbands', array([[0.7161681]])), ('people', array([[0.69913829]])), ('boyfriend', array([[0.69245769]])), ('spouses', array([[0.69183881]]))]
----------------------------------

In [218]:
sim_test(['woman', 'sibling'], 'sister')

woman, sibling -> sister
GPT-3:
0.8481172800513743
------------------------------------------------------
OPT:
0.7058625376889611
------------------------------------------------------


In [219]:
sim_test(['women', 'siblings'], 'sisters')

women, siblings -> sisters
GPT-3:
0.8795312896585115
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'women']
0.7535168206827803
------------------------------------------------------


In [220]:
define(['girl', 'sibling'])

girl, sibling
GPT-3:
[('girl', array([[0.9459775]])), ('sibling', array([[0.94597749]])), ('window', array([[0.91540565]])), ('use', array([[0.91085134]])), ('primary', array([[0.90949964]])), ('unfortunately', array([[0.90856722]])), ('sight', array([[0.90597707]])), ('supply', array([[0.90581062]])), ('novel', array([[0.90564322]])), ('impressed', array([[0.90547067]])), ('convention', array([[0.90505321]])), ('end', array([[0.90493376]]))]
------------------------------------------------------
OPT:
[('girl', array([[0.88919549]])), ('boyfriend', array([[0.7888657]])), ('sibling', array([[0.7615359]])), ('siblings', array([[0.75971009]])), ('sister', array([[0.75658656]])), ('boy', array([[0.74645393]])), ('sisters', array([[0.73892425]])), ('boys', array([[0.73056955]])), ('girls', array([[0.71688293]])), ('guy', array([[0.71393727]])), ('woman', array([[0.71070014]])), ('daughter', array([[0.70900997]]))]
------------------------------------------------------


In [221]:
define(['girl', 'siblings'])

girl, siblings
GPT-3:
[('girl', array([[0.95943611]])), ('siblings', array([[0.9594361]])), ('attach', array([[0.94210448]])), ('window', array([[0.93377698]])), ('painful', array([[0.92442016]])), ('instead', array([[0.92411282]])), ('palace', array([[0.92192259]])), ('confront', array([[0.92050739]])), ('essentially', array([[0.92033357]])), ('lazy', array([[0.9194402]])), ('religious', array([[0.91651251]])), ('novel', array([[0.91625012]]))]
------------------------------------------------------
OPT:
[('girl', array([[0.88526684]])), ('siblings', array([[0.82802366]])), ('boyfriend', array([[0.82448777]])), ('boys', array([[0.78791937]])), ('girls', array([[0.77979248]])), ('sisters', array([[0.77827186]])), ('daughter', array([[0.76678805]])), ('boy', array([[0.75019388]])), ('sister', array([[0.74346141]])), ('woman', array([[0.74186804]])), ('girlfriend', array([[0.7392184]])), ('brothers', array([[0.73762833]]))]
------------------------------------------------------


In [222]:
sim_test(['girl', 'sibling'], 'sister')

girl, sibling -> sister
GPT-3:
0.8665039544669283
------------------------------------------------------
OPT:
0.7565865573462401
------------------------------------------------------


In [223]:
sim_test(['girls', 'siblings'], 'sisters')

girls, siblings -> sisters
GPT-3:
0.8587090827138641
------------------------------------------------------
OPT:
0.819369708249551
------------------------------------------------------


* Female, children, daughter

In [224]:
define(['girl', 'children'])

girl, children
GPT-3:
[('children', array([[0.94094108]])), ('girl', array([[0.94094108]])), ('uncle', array([[0.91653626]])), ('adopt', array([[0.91585119]])), ('window', array([[0.91493453]])), ('star', array([[0.91407403]])), ('they', array([[0.91147709]])), ('bake', array([[0.90771697]])), ('palace', array([[0.9073263]])), ('painful', array([[0.90538119]])), ('traveller', array([[0.90279068]])), ('confront', array([[0.90249859]]))]
------------------------------------------------------
OPT:
[('children', array([[0.90536403]])), ('girl', array([[0.87788343]])), ('girls', array([[0.82511924]])), ('boyfriend', array([[0.82508051]])), ('boys', array([[0.81460421]])), ('daughter', array([[0.81100345]])), ('woman', array([[0.79871315]])), ('girlfriend', array([[0.7501949]])), ('boy', array([[0.73805745]])), ('guy', array([[0.7102405]])), ('brothers', array([[0.70649786]])), ('himself', array([[0.68645132]]))]
------------------------------------------------------


In [225]:
define(['woman', 'children'])

woman, children
GPT-3:
[('children', array([[0.9370872]])), ('woman', array([[0.93708719]])), ('adopt', array([[0.91614025]])), ('star', array([[0.9148706]])), ('uncle', array([[0.91383739]])), ('they', array([[0.91332712]])), ('palace', array([[0.91010612]])), ('environmental', array([[0.90185463]])), ('disappointment', array([[0.90089292]])), ('bake', array([[0.90038313]])), ('painful', array([[0.89864822]])), ('cancel', array([[0.89739831]]))]
------------------------------------------------------
OPT:
[('children', array([[0.91337232]])), ('woman', array([[0.90825248]])), ('daughter', array([[0.81887212]])), ('boyfriend', array([[0.77666656]])), ('girl', array([[0.7477719]])), ('girls', array([[0.74721739]])), ('girlfriend', array([[0.73548311]])), ('boys', array([[0.73020051]])), ('sheep', array([[0.71167462]])), ('himself', array([[0.70014884]])), ('people', array([[0.69824698]])), ('wives', array([[0.69449494]]))]
------------------------------------------------------


In [226]:
define(['female', 'children'])

female, children
GPT-3:
[('children', array([[0.94857671]])), ('female', array([[0.9485767]])), ('deep', array([[0.92573277]])), ('uncle', array([[0.91951944]])), ('adopt', array([[0.91738473]])), ('painful', array([[0.91609185]])), ('star', array([[0.91604889]])), ('palace', array([[0.91505008]])), ('particularly', array([[0.9142275]])), ('environmental', array([[0.91276548]])), ('bake', array([[0.91231643]])), ('they', array([[0.91155265]]))]
------------------------------------------------------
OPT:
[('children', array([[0.87996103]])), ('female', array([[0.84014377]])), ('male', array([[0.80123607]])), ('daughter', array([[0.75877657]])), ('girls', array([[0.74866693]])), ('woman', array([[0.72263096]])), ('boys', array([[0.71128611]])), ('boyfriend', array([[0.7075634]])), ('ourselves', array([[0.70732907]])), ('themselves', array([[0.70521577]])), ('herself', array([[0.68938328]])), ('yourself', array([[0.68817425]]))]
------------------------------------------------------


In [227]:
sim_test(['girl', 'chidren'], 'daughter')

girl, chidren -> daughter
GPT-3:
0.8401352909477144
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'chid', 'ren']
0.7325265578442671
------------------------------------------------------


In [228]:
sim_test(['woman', 'children'], 'daughter')

woman, children -> daughter
GPT-3:
0.8420301527910762
------------------------------------------------------
OPT:
0.8188721245126398
------------------------------------------------------


In [229]:
sim_test(['female', 'children'], 'daughter')

female, children -> daughter
GPT-3:
0.8445875597374192
------------------------------------------------------
OPT:
0.7587765659774348
------------------------------------------------------


In [230]:
define(['female', 'offspring'])

female, offspring
GPT-3:
[('offspring', array([[0.9552243]])), ('female', array([[0.95522429]])), ('international', array([[0.95321031]])), ('deep', array([[0.93202035]])), ('painful', array([[0.92778771]])), ('suited', array([[0.92689145]])), ('married', array([[0.92329431]])), ('cancel', array([[0.92124075]])), ('palace', array([[0.92106429]])), ('running', array([[0.92046663]])), ('south', array([[0.91710052]])), ('faith', array([[0.91676896]]))]
------------------------------------------------------
OPT:
[('female', array([[0.87692438]])), ('male', array([[0.84060205]])), ('offspring', array([[0.8110481]])), ('offence', array([[0.70519822]])), ('offense', array([[0.7004269]])), ('yourself', array([[0.68977235]])), ('engineer', array([[0.68897642]])), ('herself', array([[0.68803341]])), ('offsprings', array([[0.68038262]])), ('daughter', array([[0.67888336]])), ('characteristic', array([[0.67719783]])), ('theater', array([[0.67619683]]))]
--------------------------------------------

In [231]:
define(['female', 'offsprings'])

female, offsprings
GPT-3:
[('offsprings', array([[0.95897397]])), ('female', array([[0.95897397]])), ('painful', array([[0.94093495]])), ('deep', array([[0.93769553]])), ('cancel', array([[0.9321885]])), ('sir', array([[0.92848834]])), ('p.m.', array([[0.92823463]])), ('geography', array([[0.92542417]])), ('faith', array([[0.92316683]])), ('reach', array([[0.92267821]])), ('detailed', array([[0.9222622]])), ('running', array([[0.92177646]]))]
------------------------------------------------------
OPT:
[('female', array([[0.87725824]])), ('male', array([[0.83706846]])), ('offspring', array([[0.75311841]])), ('offsprings', array([[0.74021413]])), ('yourself', array([[0.67846745]])), ('herself', array([[0.67311358]])), ('offence', array([[0.66997113]])), ('engineer', array([[0.66761057]])), ('offense', array([[0.66697159]])), ('theater', array([[0.66098366]])), ('upsetting', array([[0.65994977]])), ('ourselves', array([[0.65955305]]))]
-----------------------------------------------------

In [232]:
sim_test(['female', 'offspring'], 'daughter')

female, offspring -> daughter
GPT-3:
0.8677396355942211
------------------------------------------------------
OPT:
0.678883357606557
------------------------------------------------------


In [233]:
sim_test(['female', 'offsprings'], 'daughters')

female, offsprings -> daughters
GPT-3:
0.8918549073463923
------------------------------------------------------
OPT:
0.5291348186725898
------------------------------------------------------


In [234]:
define(['woman', 'offspring'])

woman, offspring
GPT-3:
[('offspring', array([[0.95358454]])), ('woman', array([[0.95358453]])), ('international', array([[0.94209221]])), ('suited', array([[0.91873119]])), ('married', array([[0.91066208]])), ('disappointment', array([[0.90805711]])), ('movie', array([[0.90690083]])), ('palace', array([[0.90676449]])), ('south', array([[0.90427567]])), ('insert', array([[0.90362966]])), ('actor', array([[0.90271004]])), ('faith', array([[0.90241922]]))]
------------------------------------------------------
OPT:
[('woman', array([[0.91045006]])), ('offspring', array([[0.82900441]])), ('daughter', array([[0.78466042]])), ('boyfriend', array([[0.78027761]])), ('girl', array([[0.74647449]])), ('sheep', array([[0.73401525]])), ('girlfriend', array([[0.73270331]])), ('manner', array([[0.73088709]])), ('children', array([[0.72963593]])), ('husband', array([[0.7147413]])), ('moment', array([[0.70754416]])), ('grandmother', array([[0.70515811]]))]
---------------------------------------------

In [235]:
define(['woman', 'offsprings'])

woman, offsprings
GPT-3:
[('offsprings', array([[0.95075304]])), ('woman', array([[0.95075303]])), ('sir', array([[0.92102418]])), ('painful', array([[0.92080745]])), ('railroad', array([[0.91733202]])), ('cancel', array([[0.91730146]])), ('geography', array([[0.91642609]])), ('disappointment', array([[0.9160576]])), ('faith', array([[0.91517561]])), ('fetch', array([[0.91495424]])), ('remarkably', array([[0.9145772]])), ('p.m.', array([[0.91418087]]))]
------------------------------------------------------
OPT:
[('woman', array([[0.90820128]])), ('offspring', array([[0.78036752]])), ('boyfriend', array([[0.76019013]])), ('daughter', array([[0.7601111]])), ('offsprings', array([[0.75886107]])), ('girl', array([[0.73544182]])), ('manner', array([[0.72162578]])), ('girlfriend', array([[0.71212304]])), ('sheep', array([[0.71041533]])), ('children', array([[0.69923521]])), ('husband', array([[0.6961577]])), ('moment', array([[0.69528562]]))]
------------------------------------------------

In [236]:
sim_test(['woman', 'offspring'], 'daughter')

woman, offspring -> daughter
GPT-3:
0.8565423896407167
------------------------------------------------------
OPT:
0.7846604170325195
------------------------------------------------------


In [237]:
sim_test(['women', 'offsprings'], 'daughters')

women, offsprings -> daughters
GPT-3:
0.8878329752833398
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'women']
0.6137858841638983
------------------------------------------------------


### C - Animals, diminutives, and augmentatives!
Is a baby duck a duckling?

In [238]:
define(['baby', 'duck'])

baby, duck
GPT-3:
[('duck', array([[0.95701729]])), ('baby', array([[0.95701729]])), ('fetch', array([[0.94558363]])), ('p.m.', array([[0.94479164]])), ('faith', array([[0.94428225]])), ('painful', array([[0.9392233]])), ('geography', array([[0.93803537]])), ('musician', array([[0.93739275]])), ('railroad', array([[0.93677627]])), ('cancel', array([[0.93586957]])), ('service', array([[0.93541335]])), ('girlfriend', array([[0.93453179]]))]
------------------------------------------------------
OPT:
[('baby', array([[0.8944635]])), ('duckling', array([[0.82070828]])), ('duck', array([[0.73324031]])), ('dull', array([[0.63670497]])), ('boyfriend', array([[0.63140144]])), ('dog', array([[0.6271043]])), ('grandchild', array([[0.60557158]])), ('brother', array([[0.60462366]])), ('manner', array([[0.59863398]])), ('boy', array([[0.59857614]])), ('gentleman', array([[0.59816138]])), ('brothers', array([[0.59461311]]))]
------------------------------------------------------


In [239]:
sim_test(['baby', 'duck'], 'duckling')

baby, duck -> duckling
GPT-3:
0.897308247851017
------------------------------------------------------
OPT:
0.8207082787645852
------------------------------------------------------


In [240]:
define(['small', 'duck'])

small, duck
GPT-3:
[('duck', array([[0.9450144]])), ('small', array([[0.94501439]])), ('sir', array([[0.92546081]])), ('painful', array([[0.92234558]])), ('nervous', array([[0.91944586]])), ('faith', array([[0.91931763]])), ('tidy', array([[0.91709425]])), ('fold', array([[0.91632713]])), ('cancel', array([[0.91627355]])), ('actor', array([[0.91604444]])), ('true', array([[0.9158778]])), ('prize', array([[0.91558175]]))]
------------------------------------------------------
OPT:
[('small', array([[0.88045688]])), ('tiny', array([[0.75464504]])), ('large', array([[0.74117972]])), ('duckling', array([[0.74019656]])), ('duck', array([[0.67832458]])), ('dull', array([[0.67627452]])), ('big', array([[0.66218972]])), ('medium', array([[0.64528117]])), ('huge', array([[0.64096338]])), ('largely', array([[0.64061875]])), ('massive', array([[0.62553514]])), ('enormous', array([[0.61891381]]))]
------------------------------------------------------


In [241]:
sim_test(['small', 'duck'], 'duckling')

small, duck -> duckling
GPT-3:
0.882054549998347
------------------------------------------------------
OPT:
0.7401965589597839
------------------------------------------------------


In [242]:
define(['little', 'duck'])

little, duck
GPT-3:
[('duck', array([[0.95614679]])), ('little', array([[0.95614679]])), ('uncomfortable', array([[0.94179995]])), ('permit', array([[0.92754926]])), ('fetch', array([[0.92190154]])), ('faith', array([[0.92148741]])), ('painful', array([[0.92067581]])), ('railroad', array([[0.91942785]])), ('palace', array([[0.91902756]])), ('cancel', array([[0.91844589]])), ('tidy', array([[0.91806266]])), ('separately', array([[0.91772735]]))]
------------------------------------------------------
OPT:
[('little', array([[0.87876319]])), ('duckling', array([[0.72971186]])), ('duck', array([[0.68317835]])), ('tiny', array([[0.68098247]])), ('dull', array([[0.63744482]])), ('small', array([[0.59295361]])), ('big', array([[0.58550272]])), ('cheerful', array([[0.56135181]])), ('supermarket', array([[0.54468194]])), ('mysterious', array([[0.54396742]])), ('ugly', array([[0.54304636]])), ('quiet', array([[0.54301485]]))]
------------------------------------------------------


In [243]:
sim_test(['little', 'duck'], 'duckling')

little, duck -> duckling
GPT-3:
0.8837072469577396
------------------------------------------------------
OPT:
0.7297118572340415
------------------------------------------------------


In [244]:
define(['child', 'duck'])

child, duck
GPT-3:
[('duck', array([[0.95391393]])), ('child', array([[0.95391393]])), ('permit', array([[0.92738942]])), ('painful', array([[0.92647701]])), ('dollar', array([[0.92209303]])), ('faith', array([[0.92097938]])), ('traveller', array([[0.91975183]])), ('tip', array([[0.91944645]])), ('tidy', array([[0.91866874]])), ('musician', array([[0.91796413]])), ('lecture', array([[0.91770484]])), ('sensitive', array([[0.91765714]]))]
------------------------------------------------------
OPT:
[('child', array([[0.88113026]])), ('duckling', array([[0.76597676]])), ('duck', array([[0.7302207]])), ('grandchild', array([[0.66464533]])), ('dog', array([[0.64402397]])), ('kid', array([[0.62809922]])), ('dull', array([[0.61413976]])), ('children', array([[0.60097775]])), ('horse', array([[0.60066857]])), ('bird', array([[0.59539006]])), ('daughter', array([[0.59424374]])), ('boyfriend', array([[0.59198555]]))]
------------------------------------------------------


In [245]:
sim_test(['child', 'duck'], 'duckling')

child, duck -> duckling
GPT-3:
0.8923464670576384
------------------------------------------------------
OPT:
0.7659767552101429
------------------------------------------------------


In [246]:
define(['small', 'cat'])

small, cat
GPT-3:
[('cat', array([[0.95670576]])), ('small', array([[0.95670575]])), ('sir', array([[0.93894873]])), ('request', array([[0.92836981]])), ('true', array([[0.92641984]])), ('pride', array([[0.92635361]])), ('painful', array([[0.92512622]])), ('debt', array([[0.92436316]])), ('category', array([[0.92403086]])), ('prize', array([[0.92276058]])), ('nervous', array([[0.92238745]])), ('fold', array([[0.92049372]]))]
------------------------------------------------------
OPT:
[('small', array([[0.82094419]])), ('cat', array([[0.79901713]])), ('tiny', array([[0.72089972]])), ('large', array([[0.71504081]])), ('medium', array([[0.63646627]])), ('largely', array([[0.63480959]])), ('dog', array([[0.63187101]])), ('big', array([[0.62717907]])), ('horse', array([[0.62409981]])), ('massive', array([[0.61414607]])), ('huge', array([[0.61372583]])), ('bird', array([[0.61055431]]))]
------------------------------------------------------


In [247]:
sim_test(['small', 'cat'], 'kitten')

small, cat -> kitten
GPT-3:
0.8577500890852248
------------------------------------------------------
OPT:
0.5512441060839587
------------------------------------------------------


In [248]:
define(['baby', 'cat'])

baby, cat
GPT-3:
[('cat', array([[0.96769925]])), ('baby', array([[0.96769925]])), ('p.m.', array([[0.94845328]])), ('faith', array([[0.94605369]])), ('fetch', array([[0.94382014]])), ('painful', array([[0.94274814]])), ('mad', array([[0.94222608]])), ('sure', array([[0.94141385]])), ('musician', array([[0.94099478]])), ('geography', array([[0.9407841]])), ('railroad', array([[0.93749465]])), ('detailed', array([[0.93652217]]))]
------------------------------------------------------
OPT:
[('baby', array([[0.86200834]])), ('cat', array([[0.85184241]])), ('dog', array([[0.7085855]])), ('horse', array([[0.67216923]])), ('duckling', array([[0.65254134]])), ('kitten', array([[0.64655186]])), ('boyfriend', array([[0.63744312]])), ('bird', array([[0.63262869]])), ('painter', array([[0.61701269]])), ('grandchild', array([[0.61482385]])), ('girl', array([[0.61238386]])), ('daughter', array([[0.60215836]]))]
------------------------------------------------------


In [249]:
sim_test(['baby', 'cat'], 'kitten')

baby, cat -> kitten
GPT-3:
0.8382702496540664
------------------------------------------------------
OPT:
0.6465518569775568
------------------------------------------------------


In [250]:
define(['child', 'cat'])

child, cat
GPT-3:
[('cat', array([[0.96481342]])), ('child', array([[0.96481342]])), ('sir', array([[0.93098449]])), ('painful', array([[0.92994462]])), ('dollar', array([[0.92767769]])), ('god', array([[0.92678911]])), ('print', array([[0.92479548]])), ('tip', array([[0.92283918]])), ('faith', array([[0.92280647]])), ('lecture', array([[0.92239058]])), ('musician', array([[0.92158506]])), ('garden', array([[0.92085516]]))]
------------------------------------------------------
OPT:
[('cat', array([[0.86243468]])), ('child', array([[0.85685055]])), ('dog', array([[0.71574176]])), ('horse', array([[0.67127791]])), ('grandchild', array([[0.65429417]])), ('bird', array([[0.64615893]])), ('animal', array([[0.64045734]])), ('kid', array([[0.63315702]])), ('kitten', array([[0.62056295]])), ('children', array([[0.60605723]])), ('petrol', array([[0.60372918]])), ('daughter', array([[0.60089627]]))]
------------------------------------------------------


In [251]:
sim_test(['child', 'cat'], 'kitten')

child, cat -> kitten
GPT-3:
0.840062334931147
------------------------------------------------------
OPT:
0.6205629534031849
------------------------------------------------------


In [252]:
define(['small', 'dog'])

small, dog
GPT-3:
[('dog', array([[0.94822268]])), ('small', array([[0.94822267]])), ('pink', array([[0.92265997]])), ('thus', array([[0.91883569]])), ('debt', array([[0.91490908]])), ('sir', array([[0.91471006]])), ('our', array([[0.91285894]])), ('anger', array([[0.9113272]])), ('painful', array([[0.91106002]])), ('faith', array([[0.91100371]])), ('project', array([[0.91087175]])), ('prize', array([[0.91061859]]))]
------------------------------------------------------
OPT:
[('small', array([[0.82949888]])), ('dog', array([[0.80736465]])), ('tiny', array([[0.72037022]])), ('large', array([[0.71748746]])), ('big', array([[0.66850792]])), ('largely', array([[0.65091305]])), ('horse', array([[0.65057691]])), ('medium', array([[0.64333134]])), ('duckling', array([[0.63044234]])), ('huge', array([[0.62830195]])), ('massive', array([[0.62361093]])), ('bird', array([[0.61673953]]))]
------------------------------------------------------


In [253]:
sim_test(['small', 'dog'], 'puppy')

small, dog -> puppy
GPT-3:
0.8379980894460466
------------------------------------------------------
OPT:
0.4666601571108455
------------------------------------------------------


In [254]:
define(['baby', 'dog'])

baby, dog
GPT-3:
[('dog', array([[0.95563051]])), ('baby', array([[0.9556305]])), ('faith', array([[0.94048947]])), ('p.m.', array([[0.9363032]])), ('fetch', array([[0.93497318]])), ('sure', array([[0.93322338]])), ('musician', array([[0.93261014]])), ('painful', array([[0.93248474]])), ('project', array([[0.93084438]])), ('mad', array([[0.93082802]])), ('pop', array([[0.93043548]])), ('thus', array([[0.92975692]]))]
------------------------------------------------------
OPT:
[('baby', array([[0.8778958]])), ('dog', array([[0.86807916]])), ('horse', array([[0.69111261]])), ('boyfriend', array([[0.68431885]])), ('duckling', array([[0.6759419]])), ('cat', array([[0.66298348]])), ('girl', array([[0.65390101]])), ('boy', array([[0.63962768]])), ('grandchild', array([[0.63870288]])), ('sheep', array([[0.63338643]])), ('bird', array([[0.63291151]])), ('daughter', array([[0.63268127]]))]
------------------------------------------------------


In [255]:
sim_test(['baby', 'dog'], 'puppy')

baby, dog -> puppy
GPT-3:
0.838821379081278
------------------------------------------------------
OPT:
0.5681281718151536
------------------------------------------------------


In [256]:
define(['child', 'dog'])

child, dog
GPT-3:
[('dog', array([[0.95146621]])), ('child', array([[0.95146621]])), ('thus', array([[0.92760923]])), ('pink', array([[0.92375225]])), ('our', array([[0.92352074]])), ('god', array([[0.92351801]])), ('painful', array([[0.92072344]])), ('anger', array([[0.92014789]])), ('faith', array([[0.91816297]])), ('jacket', array([[0.91728785]])), ('print', array([[0.91682653]])), ('lecture', array([[0.91574212]]))]
------------------------------------------------------
OPT:
[('dog', array([[0.87842071]])), ('child', array([[0.87441113]])), ('horse', array([[0.69044733]])), ('grandchild', array([[0.67781907]])), ('cat', array([[0.66894425]])), ('kid', array([[0.66740925]])), ('animal', array([[0.65607985]])), ('boyfriend', array([[0.64621574]])), ('bird', array([[0.64601425]])), ('children', array([[0.64470104]])), ('daughter', array([[0.63192984]])), ('woman', array([[0.62751116]]))]
------------------------------------------------------


In [257]:
sim_test(['child', 'dog'], 'puppy')

child, dog -> puppy
GPT-3:
0.8626987176526459
------------------------------------------------------
OPT:
0.5489872908962015
------------------------------------------------------


In [258]:
define(['baby', 'male', 'horse'])

baby, male, horse
GPT-3:
[('baby', array([[0.95063693]])), ('male', array([[0.94874088]])), ('running', array([[0.94181219]])), ('p.m.', array([[0.93671669]])), ('geography', array([[0.93580801]])), ('regulation', array([[0.93529647]])), ('fetch', array([[0.93212432]])), ('faith', array([[0.9316958]])), ('railroad', array([[0.93142148]])), ('horse', array([[0.92950995]])), ('painful', array([[0.92885185]])), ('cancel', array([[0.92825565]]))]
------------------------------------------------------
OPT:
[('baby', array([[0.83494556]])), ('horse', array([[0.80766836]])), ('male', array([[0.80014969]])), ('female', array([[0.76371495]])), ('boyfriend', array([[0.69990002]])), ('girl', array([[0.68438179]])), ('daughter', array([[0.66976619]])), ('duckling', array([[0.66668274]])), ('dog', array([[0.65867223]])), ('gentleman', array([[0.65019298]])), ('engineer', array([[0.6480748]])), ('dozen', array([[0.64406862]]))]
------------------------------------------------------


In [259]:
define(['adult', 'female', 'horse'])

adult, female, horse
GPT-3:
[('female', array([[0.95187831]])), ('painful', array([[0.93909204]])), ('deep', array([[0.93524596]])), ('adult', array([[0.93335622]])), ('running', array([[0.93160731]])), ('cancel', array([[0.93084813]])), ('geography', array([[0.9304093]])), ('critical', array([[0.92898235]])), ('railroad', array([[0.92880537]])), ('p.m.', array([[0.92683084]])), ('palace', array([[0.92650243]])), ('regulation', array([[0.92611908]]))]
------------------------------------------------------
OPT:
[('female', array([[0.84283212]])), ('adult', array([[0.81751551]])), ('male', array([[0.81625054]])), ('horse', array([[0.77601843]])), ('fashionable', array([[0.65209794]])), ('theater', array([[0.65193443]])), ('realistic', array([[0.6518423]])), ('ordinary', array([[0.6479414]])), ('normal', array([[0.64606417]])), ('animal', array([[0.64456642]])), ('striped', array([[0.64259425]])), ('professional', array([[0.64221752]]))]
---------------------------------------------------

In [260]:
define(['adult', 'male', 'pig'])

adult, male, pig
GPT-3:
[('adult', array([[0.94270278]])), ('geography', array([[0.93096675]])), ('railroad', array([[0.92994474]])), ('pig', array([[0.92802976]])), ('p.m.', array([[0.92801637]])), ('cancel', array([[0.92651977]])), ('sir', array([[0.92555307]])), ('musician', array([[0.92453199]])), ('till', array([[0.92351806]])), ('painful', array([[0.92323255]])), ('detailed', array([[0.92273397]])), ('faith', array([[0.92191605]]))]
------------------------------------------------------
OPT:
[('male', array([[0.82866941]])), ('adult', array([[0.81233877]])), ('female', array([[0.7736721]])), ('duckling', array([[0.62896686]])), ('gentleman', array([[0.62479044]])), ('pig', array([[0.61942653]])), ('boyfriend', array([[0.61735534]])), ('pupil', array([[0.61682667]])), ('cheerful', array([[0.61140559]])), ('fashionable', array([[0.61102747]])), ('depressed', array([[0.60919503]])), ('attractive', array([[0.60733198]]))]
------------------------------------------------------


In [261]:
define(['adult', 'male', 'human'])

adult, male, human
GPT-3:
[('geography', array([[0.9348806]])), ('railroad', array([[0.93430497]])), ('adult', array([[0.93266518]])), ('fetch', array([[0.93209523]])), ('p.m.', array([[0.93102566]])), ('running', array([[0.92911551]])), ('cancel', array([[0.92828343]])), ('painful', array([[0.92750833]])), ('resident', array([[0.92656574]])), ('detailed', array([[0.9261149]])), ('faith', array([[0.92582756]])), ('musician', array([[0.92527566]]))]
------------------------------------------------------
OPT:
[('male', array([[0.85302329]])), ('human', array([[0.82744897]])), ('female', array([[0.81946735]])), ('adult', array([[0.80872708]])), ('individual', array([[0.6527067]])), ('normal', array([[0.64265617]])), ('realistic', array([[0.64177597]])), ('professional', array([[0.64122464]])), ('animal', array([[0.63904789]])), ('ordinary', array([[0.63306372]])), ('aggressive', array([[0.63157068]])), ('characteristic', array([[0.63057265]]))]
--------------------------------------------

In [262]:
define(['adult', 'female', 'human'])

adult, female, human
GPT-3:
[('female', array([[0.94632514]])), ('adult', array([[0.94075289]])), ('deep', array([[0.93716648]])), ('painful', array([[0.93648091]])), ('geography', array([[0.92967681]])), ('cancel', array([[0.92935938]])), ('human', array([[0.92926627]])), ('railroad', array([[0.92857335]])), ('palace', array([[0.92690942]])), ('till', array([[0.926251]])), ('p.m.', array([[0.92597495]])), ('yours', array([[0.92562348]]))]
------------------------------------------------------
OPT:
[('female', array([[0.85558201]])), ('male', array([[0.82612877]])), ('human', array([[0.8241804]])), ('adult', array([[0.80692311]])), ('individual', array([[0.66325365]])), ('realistic', array([[0.65662445]])), ('normal', array([[0.65522186]])), ('professional', array([[0.65363421]])), ('ordinary', array([[0.65072221]])), ('traditional', array([[0.64363186]])), ('aggressive', array([[0.64337065]])), ('unique', array([[0.64064227]]))]
------------------------------------------------------


### D - Philosophy, math, miscellaneous
The rest of our old tests.

In [263]:
define(['persuade', 'sentence'])

persuade, sentence
GPT-3:
[('persuade', array([[0.94641686]])), ('sentence', array([[0.94641686]])), ('badly', array([[0.93669279]])), ('betting', array([[0.92758232]])), ('busy', array([[0.92083611]])), ('gasoline', array([[0.9202528]])), ('reality', array([[0.91669369]])), ('itself', array([[0.91620985]])), ('song', array([[0.91607274]])), ('noticeable', array([[0.91499261]])), ('affection', array([[0.91417384]])), ('abandon', array([[0.91247439]]))]
------------------------------------------------------
OPT:
[('persuade', array([[0.83671688]])), ('sentence', array([[0.8195098]])), ('convince', array([[0.69954178]])), ('involvement', array([[0.69331195]])), ('involve', array([[0.68454357]])), ('punish', array([[0.66072247]])), ('assure', array([[0.65523364]])), ('introduce', array([[0.64389573]])), ('advise', array([[0.64319198]])), ('educate', array([[0.64089168]])), ('insulting', array([[0.63973465]])), ('investigate', array([[0.6385171]]))]
----------------------------------------

In [264]:
sim_test(['persuade', 'sentence'], 'argument')

persuade, sentence -> argument
GPT-3:
0.8632533265475226
------------------------------------------------------
OPT:
0.5547963155180089
------------------------------------------------------


In [265]:
define(['logical', 'belief'])

logical, belief
GPT-3:
[('logical', array([[0.96713349]])), ('belief', array([[0.96713349]])), ('painful', array([[0.94407583]])), ('geography', array([[0.94151192]])), ('railroad', array([[0.94109712]])), ('p.m.', array([[0.94076712]])), ('faith', array([[0.94028171]])), ('tightly', array([[0.93789961]])), ('critical', array([[0.93691529]])), ('musician', array([[0.93662476]])), ('reveal', array([[0.93628935]])), ('fetch', array([[0.93533123]]))]
------------------------------------------------------
OPT:
[('logical', array([[0.86862996]])), ('belief', array([[0.86381819]])), ('logic', array([[0.81533296]])), ('believe', array([[0.7535399]])), ('belong', array([[0.71327251]])), ('rationals', array([[0.67620464]])), ('technique', array([[0.64710893]])), ('alphabetical', array([[0.63678802]])), ('procedure', array([[0.63668094]])), ('religion', array([[0.63634938]])), ('faithful', array([[0.63272187]])), ('fundamental', array([[0.63147129]]))]
-------------------------------------------

In [266]:
sim_test(['logical', 'belief'], 'argument')

logical, belief -> argument
GPT-3:
0.9160718816942857
------------------------------------------------------
OPT:
0.5709568993043491
------------------------------------------------------


In [267]:
define(['logical', 'reason'])

logical, reason
GPT-3:
[('logical', array([[0.95688798]])), ('reason', array([[0.95688797]])), ('stock', array([[0.94497193]])), ('painful', array([[0.93865343]])), ('railroad', array([[0.93573513]])), ('critical', array([[0.93289972]])), ('geography', array([[0.93179805]])), ('p.m.', array([[0.93119304]])), ('harmless', array([[0.93011167]])), ('faith', array([[0.92962625]])), ('reveal', array([[0.92855963]])), ('musician', array([[0.92583717]]))]
------------------------------------------------------
OPT:
[('reason', array([[0.87357407]])), ('logical', array([[0.80021513]])), ('logic', array([[0.78088509]])), ('reasonably', array([[0.75641834]])), ('rationals', array([[0.72245499]])), ('meaning', array([[0.64376435]])), ('expectation', array([[0.64298202]])), ('interpretation', array([[0.64164943]])), ('consideration', array([[0.63574697]])), ('argument', array([[0.63484154]])), ('partner', array([[0.63376084]])), ('listen', array([[0.63230839]]))]
-----------------------------------

In [268]:
sim_test(['logical', 'reason'], 'argument')

logical, reason -> argument
GPT-3:
0.9002934753084291
------------------------------------------------------
OPT:
0.6348415368459757
------------------------------------------------------


In [269]:
define(['thinking', 'argument'])

thinking, argument
GPT-3:
[('argument', array([[0.96122531]])), ('thinking', array([[0.9612253]])), ('railroad', array([[0.92953899]])), ('painful', array([[0.92827987]])), ('substantially', array([[0.9282762]])), ('married', array([[0.92687983]])), ('geography', array([[0.92578606]])), ('crisp', array([[0.92554119]])), ('pressure', array([[0.92531534]])), ('faith', array([[0.92478623]])), ('p.m.', array([[0.92416276]])), ('sharp', array([[0.92372705]]))]
------------------------------------------------------
OPT:
[('thinking', array([[0.85832499]])), ('argument', array([[0.82997235]])), ('consideration', array([[0.74066288]])), ('think', array([[0.72082258]])), ('meaning', array([[0.71746699]])), ('interpretation', array([[0.70234272]])), ('argue', array([[0.68953856]])), ('suggestion', array([[0.68713442]])), ('eventually', array([[0.68542112]])), ('statement', array([[0.68029933]])), ('therefore', array([[0.67930948]])), ('threaten', array([[0.67878433]]))]
-------------------------

In [270]:
sim_test(['thinking', 'argument'], 'philosophy')

thinking, argument -> philosophy
GPT-3:
0.9012961462088043
------------------------------------------------------
OPT:
0.4444142881852201
------------------------------------------------------


In [271]:
sim_test(['thought', 'argument'], 'philosophy')

thought, argument -> philosophy
GPT-3:
0.8977197605222109
------------------------------------------------------
OPT:
0.44721401809349026
------------------------------------------------------


In [272]:
define(['true', 'belief'])

true, belief
GPT-3:
[('true', array([[0.96714533]])), ('belief', array([[0.96714533]])), ('painful', array([[0.94647005]])), ('railroad', array([[0.9413442]])), ('critical', array([[0.9404674]])), ('p.m.', array([[0.94029256]])), ('geography', array([[0.93894082]])), ('sir', array([[0.93847576]])), ('cancel', array([[0.93822778]])), ('palace', array([[0.93753308]])), ('actually', array([[0.93662061]])), ('detailed', array([[0.93502189]]))]
------------------------------------------------------
OPT:
[('true', array([[0.87764979]])), ('belief', array([[0.75873532]])), ('believe', array([[0.74446273]])), ('realize', array([[0.6886078]])), ('faithful', array([[0.66734857]])), ('aware', array([[0.65545248]])), ('realistic', array([[0.65376915]])), ('real', array([[0.62629237]])), ('delighted', array([[0.62198338]])), ('relaxed', array([[0.62174303]])), ('belong', array([[0.61320332]])), ('indeed', array([[0.61007705]]))]
------------------------------------------------------


In [273]:
sim_test(['true', 'belief'], 'knowledge')

true, belief -> knowledge
GPT-3:
0.8868240877731091
------------------------------------------------------
OPT:
0.5352893002277124
------------------------------------------------------


In [274]:
define(['spiritual', 'belief'])

spiritual, belief
GPT-3:
[('spiritual', array([[0.95417885]])), ('belief', array([[0.95417885]])), ('dot', array([[0.94073239]])), ('railroad', array([[0.91999745]])), ('painful', array([[0.91858588]])), ('actually', array([[0.91716593]])), ('critical', array([[0.91644914]])), ('cancel', array([[0.91484993]])), ('geography', array([[0.91347712]])), ('musician', array([[0.91294121]])), ('palace', array([[0.9127524]])), ('independently', array([[0.91272207]]))]
------------------------------------------------------
OPT:
[('spiritual', array([[0.87034736]])), ('belief', array([[0.83798405]])), ('faithful', array([[0.72998719]])), ('spirit', array([[0.72427947]])), ('believe', array([[0.70880214]])), ('faith', array([[0.70805509]])), ('faithfully', array([[0.64903794]])), ('rationals', array([[0.64657206]])), ('religion', array([[0.64461468]])), ('belong', array([[0.64302949]])), ('celebration', array([[0.64276058]])), ('kindness', array([[0.64170325]]))]
----------------------------------

In [275]:
sim_test(['spiritual', 'belief'], 'religion')

spiritual, belief -> religion
GPT-3:
0.8485276341928929
------------------------------------------------------
OPT:
0.6446146805508773
------------------------------------------------------


In [276]:
define(['right', 'belief'])

right, belief
GPT-3:
[('right', array([[0.94906963]])), ('belief', array([[0.94906962]])), ('painful', array([[0.91685907]])), ('actually', array([[0.91252915]])), ('curious', array([[0.91236697]])), ('tightly', array([[0.91218768]])), ('railroad', array([[0.91148723]])), ('delay', array([[0.91106328]])), ('palace', array([[0.91061818]])), ('cancel', array([[0.90986761]])), ('us', array([[0.90951625]])), ('bake', array([[0.90725356]]))]
------------------------------------------------------
OPT:
[('right', array([[0.88663246]])), ('rightly', array([[0.79997001]])), ('belief', array([[0.74495137]])), ('believe', array([[0.71396814]])), ('left', array([[0.67253461]])), ('correct', array([[0.66707518]])), ('belong', array([[0.64569427]])), ('relaxed', array([[0.62526856]])), ('delighted', array([[0.61893427]])), ('relaxing', array([[0.61631692]])), ('correctly', array([[0.61619076]])), ('offence', array([[0.60949718]]))]
------------------------------------------------------


In [277]:
sim_test(['right', 'belief'], 'moral')

right, belief -> moral
GPT-3:
0.8486567301185884
------------------------------------------------------
OPT:
0.40070703800869045
------------------------------------------------------


In [278]:
sim_test(['right', 'belief'], 'ideal')

right, belief -> ideal
GPT-3:
0.8721914049388988
------------------------------------------------------
OPT:
0.5667773656074907
------------------------------------------------------


In [279]:
define(['good', 'evil', 'belief'])

good, evil, belief
GPT-3:
[('belief', array([[0.94307681]])), ('palace', array([[0.93246121]])), ('good', array([[0.9308848]])), ('painful', array([[0.92840944]])), ('cancel', array([[0.9245966]])), ('critical', array([[0.92452661]])), ('become', array([[0.92386925]])), ('phase', array([[0.92244803]])), ('other', array([[0.92232604]])), ('evil', array([[0.92223569]])), ('railroad', array([[0.92213277]])), ('harmless', array([[0.92204286]]))]
------------------------------------------------------
OPT:
[('good', array([[0.83703678]])), ('goods', array([[0.81345991]])), ('evil', array([[0.78717757]])), ('goodbye', array([[0.77193636]])), ('bad', array([[0.74157314]])), ('great', array([[0.70718767]])), ('bad-tempered', array([[0.70472051]])), ('powerful', array([[0.69615658]])), ('outstanding', array([[0.6950207]])), ('faithful', array([[0.68686234]])), ('cool', array([[0.6843338]])), ('excellent', array([[0.68182784]]))]
------------------------------------------------------


In [280]:
sim_test(['good', 'evil', 'belief'], 'moral')

good, evil, belief -> moral
GPT-3:
0.8641716196507854
------------------------------------------------------
OPT:
0.4743267906423037
------------------------------------------------------


In [281]:
sim_test(['woman', 'leader'], 'chairwoman')

woman, leader -> chairwoman
GPT-3:
0.8615575969668379
------------------------------------------------------
OPT:
0.7556887174927774
------------------------------------------------------


In [282]:
sim_test(['man', 'leader'], 'chairman')

man, leader -> chairman
GPT-3:
0.8812164216505443
------------------------------------------------------
OPT:
0.699520369640954
------------------------------------------------------


In [283]:
define(['rectangular', 'table', 'numbers'])

rectangular, table, numbers
GPT-3:
[('numbers', array([[0.94657369]])), ('table', array([[0.94514559]])), ('palace', array([[0.94003799]])), ('damage', array([[0.93838589]])), ('they', array([[0.93075692]])), ('rectangular', array([[0.9291309]])), ('us', array([[0.9284431]])), ('phase', array([[0.92728538]])), ('virtually', array([[0.92587001]])), ('my', array([[0.92576034]])), ('belief', array([[0.92559076]])), ('art', array([[0.92423126]]))]
------------------------------------------------------
OPT:
[('tablet', array([[0.82963304]])), ('table', array([[0.80849223]])), ('rectangular', array([[0.78385821]])), ('numbers', array([[0.74694204]])), ('alphabetical', array([[0.70007471]])), ('alphabetically', array([[0.69062785]])), ('cupboard', array([[0.68655391]])), ('envelope', array([[0.67585029]])), ('thickness', array([[0.67247592]])), ('arrangement', array([[0.66626549]])), ('plate', array([[0.66115098]])), ('powerset', array([[0.66048327]]))]
---------------------------------------

In [284]:
define(['relation', 'two', 'sets'])

relation, two, sets
GPT-3:
[('relation', array([[0.94488525]])), ('two', array([[0.94335603]])), ('sets', array([[0.94308371]])), ('painful', array([[0.94263134]])), ('musician', array([[0.93675604]])), ('sharp', array([[0.93627486]])), ('phase', array([[0.93601383]])), ('motor', array([[0.93586518]])), ('traveller', array([[0.93518029]])), ('faith', array([[0.93485898]])), ('critical', array([[0.93468944]])), ('reveal', array([[0.93431214]]))]
------------------------------------------------------
OPT:
[('two', array([[0.80209866]])), ('sets', array([[0.78173972]])), ('three', array([[0.77964713]])), ('several', array([[0.77486788]])), ('dozen', array([[0.74944074]])), ('four', array([[0.73844933]])), ('few', array([[0.73840925]])), ('fourteen', array([[0.72973883]])), ('characteristic', array([[0.72026034]])), ('fourteenth', array([[0.71767809]])), ('relation', array([[0.71510071]])), ('eight', array([[0.71316496]]))]
------------------------------------------------------


In [285]:
define(['scalar', 'function', 'matrix'])

scalar, function, matrix
GPT-3:
[('function', array([[0.94724589]])), ('scalar', array([[0.93606084]])), ('travel', array([[0.93469953]])), ('additional', array([[0.93334434]])), ('prison', array([[0.93159487]])), ('uncertain', array([[0.93080937]])), ('matrix', array([[0.93058896]])), ('boat', array([[0.92880126]])), ('traveller', array([[0.92601973]])), ('pressure', array([[0.92336992]])), ('late', array([[0.92257613]])), ('difficult', array([[0.92192047]]))]
------------------------------------------------------
OPT:
[('function', array([[0.80147669]])), ('matrix', array([[0.77200039]])), ('scalar', array([[0.70157783]])), ('mathematics', array([[0.66953431]])), ('procedure', array([[0.6615268]])), ('powerset', array([[0.66072817]])), ('output', array([[0.6524851]])), ('method', array([[0.64421531]])), ('determinant', array([[0.64264471]])), ('alphabetical', array([[0.64126842]])), ('calculation', array([[0.64007909]])), ('tablet', array([[0.64002983]]))]
---------------------------

In [286]:
sim_test(['objective'], 'goal')

objective -> goal
GPT-3:
0.8408090139690734
------------------------------------------------------
OPT:
0.5550858586958061
------------------------------------------------------


In [287]:
sim_test(['actor'], 'actress')

actor -> actress
GPT-3:
0.8744601066235698
------------------------------------------------------
OPT:
0.8003212504922155
------------------------------------------------------


In [288]:
sim_test(['afternoon'], 'morning')

afternoon -> morning
GPT-3:
0.8443593299688374
------------------------------------------------------
OPT:
0.6570280926965197
------------------------------------------------------


## Define Helpers (for magnitude)

In [16]:
# this function doesn't allow model specification, as GPT-3 vectors are normalized.
def sum_complexity(word, vec=False):
    """
    Args:
        word: iterable, string, or vector   -> embedding to be summed.
        vec: bool                           -> True if word is already vector.
    Returns:
        Float score of the sum of the absolute value of each datapoint in the embedding.
    """
    phrase = word
    if not vec:
        if isinstance(word, str):
            phrase = positive([word], model='opt')
        else:
            phrase = positive(word, model='opt')
    
    modified = [abs(float(x)) for x in phrase[0]]
    return sum(modified)
    

In [17]:
def above_zero_complexity(word, vec=False, threshold=0.3):
    """
    Args:
        word: iterable, string, or vector -> embedding to be evaluated
        vec: bool                         -> True if word is already vector.
        threshold: float                  -> Absolute value cutoff point
    Returns:
        Float score of the amount of 'signifigant' features according to the threshold,
        divided by the number of total dimensions.
    """
    phrase = word
    if not vec:
        if isinstance(word, str):
            phrase = positive([word], model='opt')
        else:
            phrase = positive(word, model='opt')

    modified = [abs(float(x)) for x in phrase[0] if abs(float(x)) > threshold]
    return len(modified) / phrase.shape[1]

In [18]:
def compare_complexity(words, threshold=0.3):
    """
    Args:
        words: iterable     -> items to be converted to tokens and compared.
        threshold: fload    -> threshold for signifigant features with a_z_c()
    Returns:
        None. Formatted printing results for each item in words.
    """
    for w in words:
        print(w)
        print(f"Sum Complexity: {sum_complexity(w)}")
        print(f"Above Zero Complexity t-[{threshold}]: {above_zero_complexity(w, threshold=threshold)}")
        print('------------------------------------------------------')

## Conduct Magnitude Tests
This is a shorter section, and all of our tests are on OPT-1.3b.

In [292]:
compare_complexity(['man', 'bachelor', 'husband', 'father'])

man
Sum Complexity: 3869.5862306837994
Above Zero Complexity t-[0.3]: 0.88427734375
------------------------------------------------------
bachelor
Sum Complexity: 2698.8530232064036
Above Zero Complexity t-[0.3]: 0.83837890625
------------------------------------------------------
husband
Sum Complexity: 3166.152847165996
Above Zero Complexity t-[0.3]: 0.86865234375
------------------------------------------------------
father
Sum Complexity: 3597.1006672526614
Above Zero Complexity t-[0.3]: 0.88232421875
------------------------------------------------------


In [293]:
compare_complexity(['woman', 'wife', 'mother'])

woman
Sum Complexity: 3872.5401260519952
Above Zero Complexity t-[0.3]: 0.89453125
------------------------------------------------------
wife
Sum Complexity: 3150.2648618432054
Above Zero Complexity t-[0.3]: 0.86572265625
------------------------------------------------------
mother
Sum Complexity: 3577.5514898506563
Above Zero Complexity t-[0.3]: 0.88623046875
------------------------------------------------------


In [294]:
compare_complexity(['cat', 'kitten'])

cat
Sum Complexity: 3582.265500087301
Above Zero Complexity t-[0.3]: 0.8984375
------------------------------------------------------
kitten
Sum Complexity: 2721.9515306861103
Above Zero Complexity t-[0.3]: 0.8505859375
------------------------------------------------------


In [295]:
compare_complexity(['dog', 'puppy'])

dog
Sum Complexity: 3529.758090367607
Above Zero Complexity t-[0.3]: 0.87548828125
------------------------------------------------------
puppy
Sum Complexity: 2501.963118016458
Above Zero Complexity t-[0.3]: 0.8310546875
------------------------------------------------------


In [296]:
compare_complexity(['duck', 'duckling'])

duck
Sum Complexity: 2385.5241269427966
Above Zero Complexity t-[0.3]: 0.83203125
------------------------------------------------------
duckling
Sum Complexity: 2413.2875814224694
Above Zero Complexity t-[0.3]: 0.822265625
------------------------------------------------------


In [297]:
compare_complexity(['set', 'powerset'])

set
Sum Complexity: 3559.938992388498
Above Zero Complexity t-[0.3]: 0.8818359375
------------------------------------------------------
powerset
Sum Complexity: 2987.547173115257
Above Zero Complexity t-[0.3]: 0.861328125
------------------------------------------------------


In [298]:
compare_complexity(['matrix', 'determinant'])

matrix
Sum Complexity: 2349.8595615273
Above Zero Complexity t-[0.3]: 0.83349609375
------------------------------------------------------
determinant
Sum Complexity: 2404.382755302548
Above Zero Complexity t-[0.3]: 0.814453125
------------------------------------------------------


## Phrase comparison
In the past, we have not had access to models that make a phrase, so we use positives. I will explore some of those combinations below.
Phrases of interest:
* "white house"
* "baby duck"
* "logical belief"

and so on...

In [299]:
define(['logical belief']) # this usage should work.

logical belief
GPT-3:
[('spoken', array([[0.93689853]])), ('lorry', array([[0.92014554]])), ('up', array([[0.91018386]])), ('bedroom', array([[0.8914305]])), ('sea', array([[0.8895526]])), ('list', array([[0.86985034]])), ('finished', array([[0.86606042]])), ('wife', array([[0.86285496]])), ('technology', array([[0.85796905]])), ('broken', array([[0.85456666]])), ('palace', array([[0.85325309]])), ('crack', array([[0.85229479]]))]
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'log', 'ical', ' belief']
[('logical', array([[0.90879445]])), ('logic', array([[0.82582437]])), ('rationals', array([[0.70750108]])), ('fundamental', array([[0.63134636]])), ('interpretation', array([[0.62565902]])), ('characteristic', array([[0.62406868]])), ('alphabetical', array([[0.6218814]])), ('belief', array([[0.62176686]])), ('argument', array([[0.61687262]])), ('technique', array([[0.61575624]])), ('determination', array([[0.61146013]])), ('criticism', array([[0.6

In [300]:
sim_test(["white", "house"], 'white house') # this is also valid!

white, house -> white house
GPT-3:
0.8369098553623789
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'white', ' house']
0.8336804357490843
------------------------------------------------------


In [301]:
sim_test(['white house'], 'Joe Biden') # we can even do unique, new tests with our wrappers!

white house -> Joe Biden
GPT-3:
0.8575577085369428
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'white', ' house']
Tokens Requested:
['</s>', 'Joe', ' Biden']
0.3369971
------------------------------------------------------


In [302]:
define(['baby duck'])

baby duck
GPT-3:
[('elect', array([[0.93903434]])), ('pipe', array([[0.92644236]])), ('ink', array([[0.90906323]])), ('federal', array([[0.87660101]])), ('will', array([[0.87258577]])), ('electrical', array([[0.86769641]])), ('purpose', array([[0.86710025]])), ('give', array([[0.86388197]])), ('meat', array([[0.86152264]])), ('birth', array([[0.86036832]])), ('kitten', array([[0.85628729]])), ('underneath', array([[0.8552504]]))]
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'baby', ' duck']
[('baby', array([[0.88773087]])), ('duckling', array([[0.73983023]])), ('boyfriend', array([[0.6958212]])), ('daughter', array([[0.68505451]])), ('girl', array([[0.64308662]])), ('sheep', array([[0.6389099]])), ('granddaughter', array([[0.63541719]])), ('grandson', array([[0.63522208]])), ('chairwoman', array([[0.63346683]])), ('grandchild', array([[0.63198318]])), ('horse', array([[0.63164074]])), ('chairman', array([[0.63016237]]))]
-----------------------

In [303]:
sim_test(['baby', 'duck'], 'baby duck')

baby, duck -> baby duck
GPT-3:
0.801534060134556
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'baby', ' duck']
0.8855328136843644
------------------------------------------------------


In [304]:
sim_test(['baby duck'], 'duckling')

baby duck -> duckling
GPT-3:
0.7931393521274706
------------------------------------------------------
OPT:
Tokens Requested:
['</s>', 'baby', ' duck']
0.7398302282765973
------------------------------------------------------


As some brief discussion before submit, some of these scores are very surprising. That final sim_test has me scratching my head a bit, why did OPT score it at 0.19?

There are different sizes of OPT to be tried, but I like the 2048-D vectors from OPT-1.3B, and it didn't use too much cache space. We can (and should) test this on larger versions (Paramenter size can be changed- ['125m', '350m', '1.3b', '2.7b', '6.7b', '13b', '30b']). For reference, 350M has 512-D vectors. If there is an error in vector generation I will fix it, I hope this is easy to use and read.

Notes:
* Average Similarity? How does this look, how can we calculate
* Use larger OPT models? Cache them?
* Phrase tests :)
    * sentences?

Early development and language development - which words and concepts come first?

In [24]:
def phrase_comparison(phrases):
    """
    Args:
        phrases: iterable   -> iterable of novel phrases to compare.
    Returns:
        None. Formats and facilitates phrase comparison of an iterable of novel phrases.
    """
    enc = [(positive([str(x)], 'gpt'), positive([str(x)], 'opt')) for x in phrases]
    print(*phrases, sep=', ')
    for i in range(len(phrases)):
        for j in range(i+1, len(phrases)):
            print(f"{phrases[i]} -> {phrases[j]}")
            print(f"GPT-3: {cosine_similarity(enc[i][0], enc[j][0])[0][0]}")
            print(f"OPT-1.3B: {cosine_similarity(enc[i][1], enc[j][1])[0][0]}")
            print('------------------------------------------------------')
    print("\nPhrase Definitions:")
    for p in phrases:
        define([p])

### A - spouse phrases

In [27]:
phrase_comparison(['unmarried man', 'a bachelor is an unmarried man'])

unmarried man, a bachelor is an unmarried man
unmarried man -> a bachelor is an unmarried man
GPT-3: 0.9268371372784421
OPT-1.3B: 0.595645546913147
------------------------------------------------------

Phrase Definitions:
unmarried man
GPT-3:
[('tight', 0.9585651798672634), ('hesitate', 0.904121945710308), ('around', 0.9003319095666844), ('first', 0.8971223413932642), ('thoroughly', 0.8904060820582973), ('bitter', 0.8844611505296589), ('tough', 0.878019399765661), ('temporarily', 0.8745131826127558), ('royal', 0.8744712565230287), ('many', 0.874208621756656), ('terrible', 0.8727524283726964), ('team', 0.8719793643100232)]
------------------------------------------------------
OPT:
[('unmarried', 0.9031768567209886), ('unemployed', 0.8245551567531166), ('unsuccessful', 0.7371521342852776), ('unconscious', 0.7322928921840312), ('unhappy', 0.7198059944120146), ('unimportant', 0.7096540746999525), ('unacceptable', 0.7031870338191895), ('university', 0.7017153929167079), ('unwillingly', 0

In [28]:
phrase_comparison(['adult female horse', 'Look, a mare!'])

adult female horse, Look, a mare!
adult female horse -> Look, a mare!
GPT-3: 0.8950975294257666
OPT-1.3B: 0.49122458696365356
------------------------------------------------------

Phrase Definitions:
adult female horse
GPT-3:
[('cooking', 0.9042642865598103), ('spring', 0.8568780319593201), ('recommend', 0.8414691020080547), ('essential', 0.837193505712824), ('explode', 0.8368366523571928), ('ear', 0.8360628245519418), ('enthusiastic', 0.83309916802412), ('fund', 0.8271051059598724), ('touch', 0.8221076342147099), ('action', 0.8220295040579576), ('bandage', 0.8184311276967471), ('silent', 0.818179256273843)]
------------------------------------------------------
OPT:
[('adult', 0.8141188865447431), ('female', 0.7818204378732574), ('male', 0.7615716641989176), ('daughter', 0.6636983684521554), ('horse', 0.6575904427367566), ('boyfriend', 0.6523048186683067), ('woman', 0.6381340083313645), ('girlfriend', 0.6311426784557843), ('children', 0.6247547604284629), ('regularly', 0.62337459823

In [29]:
phrase_comparison(['What do you call an unmarried man?', 'What do you call a married man?'])

What do you call an unmarried man?, What do you call a married man?
What do you call an unmarried man? -> What do you call a married man?
GPT-3: 0.9269945921124785
OPT-1.3B: 0.9753905534744263
------------------------------------------------------

Phrase Definitions:
What do you call an unmarried man?
GPT-3:
[('tight', 0.8610593566862468), ('around', 0.8254371848537954), ('reality', 0.8154799775049557), ('tough', 0.8103635927651398), ('many', 0.8008065555810218), ('royal', 0.7977282486680517), ('hesitate', 0.7951773279190766), ('sentence', 0.7939977680584587), ('bachelor', 0.7919560784166031), ('reasonably', 0.7909655678490519), ('beef', 0.7906515241797019), ('team', 0.78927766631735)]
------------------------------------------------------
OPT:
[('whereas', 0.7408555338571827), ('including', 0.7236769864996395), ('because', 0.7058284114864697), ('excluding', 0.7041162229543407), ('forget', 0.6986949543956237), ('except', 0.6974921431449932), ('called', 0.6967679959606097), ('either', 

In [31]:
phrase_comparison(['My dog had babies!', 'Do you want to play with a puppy?'])

My dog had babies!, Do you want to play with a puppy?
My dog had babies! -> Do you want to play with a puppy?
GPT-3: 0.8485986163569736
OPT-1.3B: 0.6416599750518799
------------------------------------------------------

Phrase Definitions:
My dog had babies!
GPT-3:
[('federal', 0.865685813991274), ('favour', 0.8615180869645676), ('birth', 0.8409815556608855), ('pipe', 0.8403259053782138), ('touch', 0.8329189682363651), ('kitten', 0.8293582471139342), ('give', 0.8277779903674067), ('electrical', 0.826947899486911), ('knit', 0.8188127144772351), ('essential', 0.8181095400645813), ('guess', 0.8164957123731774), ('gift', 0.8143541639446943)]
------------------------------------------------------
OPT:
[('myself', 0.6384429664598332), ('yesterday', 0.5888885300861877), ('i.e.', 0.5816660932510305), ('and', 0.57816227266013), ('afterwards', 0.5770348674711697), ('yeah', 0.5759295198028392), ('either', 0.5739917833388397), ('including', 0.5709369120982948), ('probably', 0.5683962866411623), (

In [39]:
phrase_comparison([
    'Knowledge is a true belief.', 
    'An argument is a logical belief.',
    'A spiritual belief is religion.'
])

Knowledge is a true belief., An argument is a logical belief., A spiritual belief is religion.
Knowledge is a true belief. -> An argument is a logical belief.
GPT-3: 0.8585281365656691
OPT-1.3B: 0.5583149790763855
------------------------------------------------------
Knowledge is a true belief. -> A spiritual belief is religion.
GPT-3: 0.8706939048506915
OPT-1.3B: 0.6498346328735352
------------------------------------------------------
An argument is a logical belief. -> A spiritual belief is religion.
GPT-3: 0.8733140441620892
OPT-1.3B: 0.6986019015312195
------------------------------------------------------

Phrase Definitions:
Knowledge is a true belief.
GPT-3:
[('ending', 0.8724948515377793), ('lorry', 0.867368557159757), ('sea', 0.842950573668344), ('pause', 0.8384210788576483), ('favourite', 0.8313689633450653), ('technology', 0.823188656851715), ('purchase', 0.8216901793961465), ('list', 0.8211560180418986), ('understanding', 0.8190393648773264), ('melt', 0.8180555008323629),

## Average Similarity
What is the average similarity between words in our dictionary for each model?

In [15]:
def make_avg_tbl(model='opt', extra_vocab=[]):
    """
    Args:
        model: 'gpt' or 'opt'
        extra_vocab: iterable of extra strings to be processed like vocab
    Returns:
        Table that contains the average cosine similarity of each word in our vocabulary
        to each other word, possibly including user supplied extra vocab.
    """
    print(f"Making average table for model: {model}")
    tbl_vocab = [str(x) for x in vocab + list(extra_vocab)]
    tbl_sz = len(tbl_vocab)
    table = {k: 0.0 for k in tbl_vocab}
    for term_a in tbl_vocab:
        a = positive([term_a], model)
        for term_b in tbl_vocab:
            if term_a != term_b:
                b = positive([term_b], model)
                table[term_a] += cosine_similarity(a, b)[0][0]
        table[term_a] /= tbl_sz
    print('done')
    return table                

In [16]:
def calc_avg_sim():
    # Does this need a header? Wrapper.
    print("What is our average connection strength?")
    gpt_tbl = make_avg_tbl(model='gpt')
    opt_tbl = make_avg_tbl()
    sz = len(opt_tbl)
    print("GPT-3:")
    print(sum(gpt_tbl.values()) / sz)
    print('------------------------------------------------------')
    print('OPT-1.3B:')
    print(sum(opt_tbl.values()) / sz)
    print('------------------------------------------------------')

In [17]:
calc_avg_sim()
# takes ~200m

What is our average connection strength?
Making average table for model: gpt
done
Making average table for model: opt
done
GPT-3:
0.8208506855698428
------------------------------------------------------
OPT-1.3B:
0.37241835117426914
------------------------------------------------------
